<a href="https://colab.research.google.com/github/AndreHadjiThomas/54247538/blob/main/Pipeline_NASA/GDDP-CMIP6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
pip install earthengine-api

In [69]:
pip install rasterio

In [58]:
import ee
from google.colab import auth
auth.authenticate_user()  # Authenticate with your Google account
ee.Initialize(project='ultimate-result-436120-p8')           # Initialize Earth Engine

In [56]:
# Define the boundary box for Montreal region
boundary_box = ee.Geometry.Rectangle([-74.0, 45.3, -73.4, 45.7])

# Define the ImageCollection for NEX-GDDP-CMIP6 filtered by CanESM5 model and SSP245 scenario
dataset = ee.ImageCollection('NASA/GDDP-CMIP6') \
    .filter(ee.Filter.date('2016-01-01', '2100-12-31')) \
    .filter(ee.Filter.eq('model', 'CanESM5')) \
    .filter(ee.Filter.eq('scenario', 'ssp245'))

# Select the variables: tasmax, tasmin, and pr
tasmax = dataset.select('tasmax')  # Maximum temperature
tasmin = dataset.select('tasmin')  # Minimum temperature
precipitation = dataset.select('pr')  # Precipitation

# Calculate the mean for each variable
tasmax_mean = tasmax.mean()
tasmin_mean = tasmin.mean()
precipitation_mean = precipitation.mean()

# Export tasmax (maximum temperature) to Google Drive
task_tasmax = ee.batch.Export.image.toDrive(
    image=tasmax_mean,
    description='NEX_GDDP_CMIP6_Tasmax_CanESM5_SSP245',
    fileFormat='GeoTIFF',
    region=boundary_box,
    scale=10000
)
task_tasmax.start()

# Export tasmin (minimum temperature) to Google Drive
task_tasmin = ee.batch.Export.image.toDrive(
    image=tasmin_mean,
    description='NEX_GDDP_CMIP6_Tasmin_CanESM5_SSP245',
    fileFormat='GeoTIFF',
    region=boundary_box,
    scale=10000
)
task_tasmin.start()

# Export pr (precipitation) to Google Drive
task_pr = ee.batch.Export.image.toDrive(
    image=precipitation_mean,
    description='NEX_GDDP_CMIP6_Precipitation_CanESM5_SSP245',
    fileFormat='GeoTIFF',
    region=boundary_box,
    scale=10000
)
task_pr.start()

print("Exporting tasmax, tasmin, and precipitation to Google Drive...")



Exporting tasmax, tasmin, and precipitation to Google Drive...


In [70]:
import rasterio
import xarray as xr
import numpy as np

# Define paths to the GeoTIFF files in your Google Drive
tasmax_path = '/content/drive/MyDrive/NEX_GDDP_CMIP6_Tasmax_CanESM5_SSP245.tif'
tasmin_path = '/content/drive/MyDrive/NEX_GDDP_CMIP6_Tasmin_CanESM5_SSP245.tif'
precip_path = '/content/drive/MyDrive/NEX_GDDP_CMIP6_Precipitation_CanESM5_SSP245.tif'

# Convert each GeoTIFF to NetCDF
def geotiff_to_netcdf(tiff_path, netcdf_path):
    with rasterio.open(tiff_path) as src:
        array = src.read(1)  # Read the first band (e.g., temperature or precipitation data)

        # Get the coordinates of the array using the affine transformation
        transform = src.transform
        ny, nx = array.shape
        x_coords, y_coords = np.meshgrid(np.arange(nx), np.arange(ny))
        lon, lat = rasterio.transform.xy(transform, y_coords, x_coords)

        # Convert to numpy arrays
        lon = np.array(lon)
        lat = np.array(lat)

    # Create an xarray DataArray and convert it to NetCDF
    data = xr.DataArray(array, dims=['lat', 'lon'], coords={'lat': lat[:,0], 'lon': lon[0,:]})
    data.to_netcdf(netcdf_path)

# Convert tasmax to NetCDF
geotiff_to_netcdf(tasmax_path, '/content/drive/MyDrive/NEX_GDDP_CMIP6_Tasmax_CanESM5_SSP245.nc')

# Convert tasmin to NetCDF
geotiff_to_netcdf(tasmin_path, '/content/drive/MyDrive/NEX_GDDP_CMIP6_Tasmin_CanESM5_SSP245.nc')

# Convert precipitation to NetCDF
geotiff_to_netcdf(precip_path, '/content/drive/MyDrive/NEX_GDDP_CMIP6_Precipitation_CanESM5_SSP245.nc')


In [71]:
import xarray as xr

# Open the NetCDF file
ds = xr.open_dataset('/content/drive/MyDrive/NEX_GDDP_CMIP6_Tasmax_CanESM5_SSP245.nc')

# Inspect the dataset
#xr.set_options(display_max_rows=100)  # Set the maximum number of rows to display
print(ds)

<xarray.Dataset> Size: 236B
Dimensions:                        (lon: 7, lat: 5)
Coordinates:
  * lon                            (lon) float64 56B -73.98 -73.89 ... -73.44
  * lat                            (lat) float64 40B 45.68 45.59 ... 45.41 45.32
Data variables:
    __xarray_dataarray_variable__  (lat, lon) float32 140B ...


In [67]:
print(ds['__xarray_dataarray_variable__'].values)

[[288.19672 288.19672 288.19672 288.1335  288.1335  288.1335  287.95883]
 [288.19672 288.19672 288.19672 288.1335  288.1335  288.1335  287.95883]
 [288.27167 288.27167 288.27167 288.22522 288.22522 288.22522 288.22787]
 [288.27167 288.27167 288.27167 288.22522 288.22522 288.22522 288.22787]
 [288.27167 288.27167 288.27167 288.22522 288.22522 288.22522 288.22787]]
